In [ ]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)

import datetime as dt

In [ ]:
today = dt.datetime.now()
year = today.year

In [ ]:
# get file paths
curr_filepath = input('input file location of report 4-1: ') #
prev_filepath = input('input file location of previous report: ') #

In [ ]:
# read in data
df_curr = pd.read_excel(curr_filepath) # report 4-1
df_prev = pd.read_excel(prev_filepath) # previous report

In [ ]:
# create text file for links used
with open('direct_party_dob-doi.txt', 'w') as f:
    f.write('Current Report path: ' + curr_filepath)
    f.write('\nPrevious Report path: ' + prev_filepath)
f.close()

In [ ]:
df_curr['Batch'] = input('input batch number (eg. 2023 Q3): ')

In [ ]:
df_curr[['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']] = ""

In [ ]:
df_prev['Date of Birth']  = pd.to_datetime(df_prev['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date
df_prev['Date of Incorporation']  = pd.to_datetime(df_prev['Date of Incorporation'], format='%Y-%m-%d', errors='coerce').dt.date

df_prev['dob_year'] = df_prev['Date of Birth'].apply(lambda x: x.year)
df_prev['doi_year'] = df_prev['Date of Incorporation'].apply(lambda x: x.year)

In [ ]:
df_curr['Date of Birth']  = pd.to_datetime(df_curr['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date
df_curr['Date of Incorporation']  = pd.to_datetime(df_curr['Date of Incorporation'], format='%Y-%m-%d', errors='coerce').dt.date

df_curr['dob_year'] = df_curr['Date of Birth'].apply(lambda x: x.year)
df_curr['doi_year'] = df_curr['Date of Incorporation'].apply(lambda x: x.year)

In [ ]:
df_prev['key'] = df_prev['Portfolio Number'].astype('str') + df_prev['Party ID'].astype('str')
df_curr['key'] = df_curr['Portfolio Number'].astype('str') + df_curr['Party ID'].astype('str')

In [ ]:
# filter active status
df_curr = df_curr[df_curr['Current Status'] == 'Active']

In [ ]:
#filter dob/doi
df_curr = df_curr[(year-df_curr['dob_year']>100) | (year-df_curr['doi_year']>100)]
df_prev = df_prev[(year-df_prev['dob_year']>100) | (year-df_prev['doi_year']>100)]

In [ ]:
# identify parties that appeared in previous report
prev_parties = df_prev['key'].tolist()

In [ ]:
# identify parties that appear in current report
curr_parties = df_curr['key'].tolist()

In [ ]:
# list of parties that appear in both previous and current report (unresolved or exceptions)
both_parties = set(prev_parties).intersection(set(curr_parties))

In [ ]:
# list of new parties that appear only in the current report
new_parties = set(curr_parties).difference(set(prev_parties))

In [ ]:
# total unresolved = old unresolved (both_parties) and new unresolved (new_parties)
df_curr = df_curr[(df_curr['key'].isin(both_parties)) | (df_curr['key'].isin(new_parties))]

In [ ]:
#append df_prev['key'].isin(both_parties) and df_curr['key'].isin(new_parties)
df = pd.concat([df_prev[df_prev['key'].isin(both_parties)], df_curr[df_curr['key'].isin(new_parties)]])

In [ ]:
# drop extra columns, rearrange columns
df = df.drop(columns=['dob_year','doi_year', 'key'])

df = df[['Batch', 'Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception',
                   'Portfolio Number', 'Portfolio Name', 'Current Status', 'Relationship Type', 'Relationship Sub-Type', 
                   'RM Name', 'RM Location Name', 'RM Team Name', 'Booking Center Officer Name','Managed By EAM / EIA / Finder',
                   'EAM / EIA Name', 'Finder Name','Portfolio Opening Date', 'Relationship (Party Role)', 'Party Name',
                   'Party Type', 'Party ID', 'Is Public', 'Date of Birth','Date of Incorporation']]

In [ ]:
# print to excel
df.to_excel('direct_party_dob-doi.xlsx', index=False)